<a href="https://colab.research.google.com/github/prateekjoshi565/textrank_text_summarization/blob/master/TestRank_Text_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt') # one time execution
import re

[nltk_data] Error loading punkt: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>


In [3]:
import pandas as pd
file_path = '/home/dharmendra/Desktop/desk/captsone/senti_outputs/negative_csv.csv'
df = pd.read_csv(file_path)

In [4]:
type(df['Negative_Texts'])

pandas.core.series.Series

In [5]:
df['Negative_Texts']

0      In the 1980s I spent several summers in Italy....
1      Our cats thrive extremely well on this dry cat...
2      The item over all was fine, but the Banana Hea...
3      I have tried literally dozens of teas since be...
4      I think I have tried just about every Bloody M...
5      I've been enjoying this tea for years.  This t...
6      This is one of the best tasting crisps I have ...
7      I've tried 5-hour energy, red rain, NOS, and o...
8      If you're looking for an energy boost without ...
9      I tried to find israeli couscous in a number o...
10     I like a lot of sesame oil and use it in salad...
11     That's exactly what I was looking for to bake ...
12     Green Mountain is my favorite brand. Nantucket...
13     They aren't fattening and they're good for you...
14     Now, I haven't done a side-by-side comparison,...
15     These are so tangy it prevents me from eating ...
16     I have always been a fan of Kettle chips.  Whe...
17     I agree with the other r

In [6]:
# split the the text in the articles into sentences
sentences = []
for s in df['Negative_Texts']:
  sentences.append(sent_tokenize(s))

In [7]:
# flatten the list
sentences = [y for x in sentences for y in x]

In [8]:
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

In [9]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [11]:
# function to remove stopwords
def remove_stopwords(sen):
  sen_new = " ".join([i for i in sen if i not in stop_words])
  return sen_new

In [12]:
# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

In [21]:
# download pretrained GloVe word embeddings
! wget http://nlp.stanford.edu/data/glove.6B.zip

--2019-03-03 22:05:22--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2019-03-03 22:05:23--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.3’

glove.6B.zip.3       25%[====>               ] 206.34M   677KB/s    eta 27m 48s^C


In [22]:
!unzip glove*.zip

Archive:  glove.6B.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of glove.6B.zip or
        glove.6B.zip.zip, and cannot find glove.6B.zip.ZIP, period.


In [13]:
# Extract word vectors
word_embeddings = {}
f = open('/home/dharmendra/Desktop/textrank_text_summarization-master/glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [14]:
sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)

In [15]:
len(sentence_vectors)

1800

In [16]:
sentence_vectors[:3]

[array([-0.01628342, -0.00610871, -0.12903023,  0.07460535,  0.23960435,
         0.12024993,  0.01822994,  0.5361185 , -0.5586003 , -0.04863385,
         0.2671332 , -0.42973503, -0.23734592,  0.41628093, -0.15663384,
        -0.36158237,  0.25035498, -0.50993127, -0.33164135,  0.25691378,
         0.06292402,  0.6926119 ,  0.12224718,  0.33467036, -0.3401057 ,
        -0.45760563, -0.13419648, -0.32606098,  0.04936016,  0.14732817,
        -0.19576105,  0.17846365, -0.2927743 , -0.06607274, -0.06057411,
         0.6657236 ,  0.13440038, -0.17660335, -0.0518028 ,  0.05055236,
        -0.9462759 , -0.33350933,  0.4684194 ,  0.1291992 ,  0.31834918,
         0.06046738,  0.32429492, -0.36071733,  0.5118508 , -0.28802726,
         0.25945514, -0.35395405,  0.08218697,  0.8714846 , -0.15613349,
        -2.0462632 , -0.08798852, -0.25176704,  0.7299375 ,  0.82426393,
        -0.06750988,  1.0809398 ,  0.10132767, -0.2136416 ,  0.31857538,
         0.05500726,  0.13270432,  0.18207647,  0.4

In [17]:
sentence_vectors[0].shape

(100,)

The next step is to find similarities among the sentences. We will use cosine similarity to find similarity between a pair of sentences. Let's create an empty similarity matrix for this task and populate it with cosine similarities of the sentences.

In [18]:
# similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])

In [19]:
from sklearn.metrics.pairwise import cosine_similarity

In [20]:
for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [21]:
import networkx as nx

nx_graph = nx.from_numpy_matrix(sim_mat)
scores = nx.pagerank(nx_graph)

In [22]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

In [23]:
ranked_sentences[:5]

[(0.0006602153199527035,
  "If you want really tea that is rather sweet, you'll either have to look further or use this with added sweetener."),
 (0.0006600982860089375,
  "I mainly use my Keurig for coffee but thought I'd try this one since I love dark chocolate and it was getting mostly good reviews."),
 (0.0006594134220052381,
  "I'd just pick something else if I'm going to pound water all day long (perhaps the blueberry green tea one which is NOT sweetened - it's excellent as well)  I'll use this for one bottle and a non-sweetened one for the other."),
 (0.0006586892879758086,
  "I also noticed that you don't find nearly as much coating from flavoring on your fingers as I've experienced when eating other chips.<br /><br />Purchasing it by the case to try out all of the flavors is a good idea and a decent value."),
 (0.0006577045613059301,
  "I've done nutritional coaching for 30 years and watched healthy foods progress from painfully bad to barely edible to excellent.<br /><br />Th

In [24]:
# Specify number of sentences to form the summary
sn = 10

# Generate summary
for i in range(sn):
  print(ranked_sentences[i][1])

If you want really tea that is rather sweet, you'll either have to look further or use this with added sweetener.
I mainly use my Keurig for coffee but thought I'd try this one since I love dark chocolate and it was getting mostly good reviews.
I'd just pick something else if I'm going to pound water all day long (perhaps the blueberry green tea one which is NOT sweetened - it's excellent as well)  I'll use this for one bottle and a non-sweetened one for the other.
I also noticed that you don't find nearly as much coating from flavoring on your fingers as I've experienced when eating other chips.<br /><br />Purchasing it by the case to try out all of the flavors is a good idea and a decent value.
I've done nutritional coaching for 30 years and watched healthy foods progress from painfully bad to barely edible to excellent.<br /><br />This neanderthal-era gluten free bar is so bad I want to go to my local grocery store, take all the boxes off off the shelf and hide them in a different p